# Polygonization

In this workshop we will learn how to convert our topological model of the building into a geometric model.

## 0. Initialization

### 0.0. Importing libraries 

In [2]:
import topogenesis as tg
import numpy as np 
import pyvista as pv
import os
import trimesh as tm
import resources.boolean_marching_cubes as bmc

### 0.1. Generate Symmetry Stencils

In [34]:
# example symmetry strings
##########################
# sym_str = [["OO"], ["XP"], ["XN"], ["YP"], ["YN"], ["ZP"], ["ZN"]]
# sym_str = [["OO"], ["XX"], ["YP"], ["YN"], ["ZP"], ["ZN"]]
# sym_str = [["OO"], ["XX"], ["YY"], ["ZP"], ["ZN"]]
sym_str = [["OO"], ["XX", "YY"], ["ZZ"]]
# sym_str = [["OO"], ["XX", "YY", "ZZ"]]
# sym_str = [["OO"]]

stencils = bmc.create_symmetry_stencils(sym_str)

### 0.2. Generate lattices for all possible cubes 

In [35]:
# generate binary representation of all the possible cubes
l_bis = bmc.bi_cube_lattices()

## 1. Profiling

### 1.1. Catalogue the profile of all corners

In [36]:
# find all unique corner arrangements based on stencils
corner_profiles = bmc.extract_corner_profiles(stencils, l_bis)

### 1.2. Find unique corner profiles

In [37]:
# stack corner_profiles vertically
cp_stacked = np.vstack(corner_profiles)

# find the unique arrangements of corners
uniq_corner_arang = np.unique(cp_stacked, axis=0)

print(len(uniq_corner_arang))
print(uniq_corner_arang)

12
[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [0 2 0]
 [0 2 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]
 [1 2 0]
 [1 2 1]]


### 1.3. Construct unique profile latices

In [38]:
# construct lattices for all unique corner profiles
(corner_loc_lattices, corner_neigh_lattices) = bmc.profiles_to_lattices(uniq_corner_arang, stencils)

### 1.4. Visualize unique profiles

In [39]:
p = pv.Plotter(notebook=True)

base_lattice = corner_neigh_lattices[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    f = int(value)
    lattice = corner_neigh_lattices[f]
    loc = corner_loc_lattices[f]

    # Add the data values to the cell data
    grid.cell_arrays["filled"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="filled")
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=0.7, show_scalar_bar=False)
    
    # Add the data values to the cell data
    grid.cell_arrays["corner"] = loc.flatten(order="F").astype(int)# Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="corner")
    # adding the voxels
    p.add_mesh(threshed, name='sphere2', show_edges=True, opacity=1.0, show_scalar_bar=False, color="white")

    return

p.add_slider_widget(create_mesh, [1, len(corner_neigh_lattices)], title='Arrangements', value=1, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(4.363703305156274, 4.363703305156274, 4.363703305156274),
 (0.5, 0.5, 0.5),
 (0.0, 0.0, 1.0)]

### 1.5. Save unique arrangement profiles 

In [40]:
# save all design templates into lattice CSVs
templates_path = os.path.relpath('../data/bmc/bmc_templates')
bmc.save_design_templates(corner_loc_lattices, corner_neigh_lattices, templates_path)

## 2. Construct the tile-set

### 2.1. Load sub-tile meshes

In [41]:
# load subtile meshes
subtile_meshes = []
for c in range(len(corner_loc_lattices)):
    corner_mesh_path = os.path.relpath('../data/bmc/bmc_subtiles/t_' + f'{c:02}' + '.obj')
    corner_mesh = tm.load(corner_mesh_path)
    subtile_meshes.append(corner_mesh)

### 2.2. Combine sub-tile meshes to create tile meshes

In [42]:

def construct_tile_meshes(subtile_meshes, corner_profiles, uniq_corner_arang, corner_loc_lattices):
    tile_corner_inds = []
    for prof in corner_profiles:
        # find each corner in the list of unique corner arrangements
        corner_ind = np.array(
            [np.where((p == uniq_corner_arang).all(1)) for p in prof]).flatten()
        tile_corner_inds.append(corner_ind)

    tile_corner_inds = np.array(tile_corner_inds)
    # print(tile_corner_inds)

    ###############################

    # find unique locs

    unique_locs = [np.array(np.where(loc == 1)).flatten()
                   for loc in corner_loc_lattices]
    unique_locs = np.array(unique_locs)

    ###############################

    corner_loc = np.array(np.where(np.ones((2, 2, 2)) == 1)).T
    corner_pos = corner_loc - .5
    # print(corner_pos)

    tiles_meshes = []
    # loading meshes
    for tile in tile_corner_inds:
        last_v_count = 0
        vertice_list = []
        face_list = []
        for c_ind, pos, loc in zip(tile, corner_pos, corner_loc):

            # extract current mesh
            corner_mesh = subtile_meshes[c_ind]
            # extract the unique profile of this corner
            u_loc = unique_locs[c_ind]
            if type(corner_mesh) == tm.base.Trimesh:
                # if current profile is different than unique profile, we need a transformation, -1 will flip over that dimension and 1 will keep it the same
                trans = (u_loc - loc) * 2 + 1
                mirror_normal = u_loc - loc
                if np.any(mirror_normal < 0) : 
                    Mx = tm.transformations.reflection_matrix([0,0,0], mirror_normal)
                    vertices_transformed = tm.transformations.transform_points(corner_mesh.vertices, Mx)
                else:
                    vertices_transformed = corner_mesh.vertices
                # append the vertices
                vertice_list.append(vertices_transformed + pos)
                face_list.append(corner_mesh.faces + last_v_count)
                last_v_count += len(corner_mesh.vertices)

        vs = []
        fs = []
        if len(vertice_list):
            vs = np.vstack(vertice_list)
            fs = np.vstack(face_list)

            tile_mesh = tm.Trimesh(vs * 0.5, fs)
            tiles_meshes.append(tile_mesh)
        else:
            empty_tile = tm.creation.icosphere(subdivisions=1, radius=0.1)
            tiles_meshes.append(empty_tile)

    return tiles_meshes

In [43]:
tiles_meshes = construct_tile_meshes(subtile_meshes, corner_profiles, uniq_corner_arang, corner_loc_lattices)

### 2.3. Visualize tile meshes

In [ ]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

p = pv.Plotter(notebook=True)

base_lattice = l_bis[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5 
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit *0.5

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    i = int(value)
    mesh = tiles_meshes[i]
    lattice = l_bis[i]

    # Add the data values to the cell data
    grid.cell_arrays["cube"] = lattice.flatten(order="F").astype(int)# Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="cube")
    # adding the voxels
    p.add_mesh(threshed, name='sphere2', show_edges=True, opacity=0.2, show_scalar_bar=False, color="white")

    # adding the meshes
    p.add_mesh(tri_to_pv(mesh), color='#abd8ff', name="sphere")

    return

p.add_slider_widget(create_mesh, [0, len(tiles_meshes)], title='Tiles', value=1, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))

p.show(use_ipyvtk=True)

### 2.4. Save the tile-set

In [12]:
tiles_path = os.path.relpath('../data/bmc/bmc_tiles')
bmc.save_tile_meshes(tiles_meshes, l_bis, tiles_path)

## 3. Boolean Marching Cube

### 3.1. Load envelope lattice

In [3]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_bunny.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

### 3.2. Extract the cube lattice from the envelope lattice

In [4]:
cube_lattice = envelope_lattice.boolean_marching_cubes()

### 3.3. tile the cube lattice with a tileset

In [5]:
custom_tiles_path = os.path.relpath('../data/bmc/bmc_tiles')
chamfer_tiles_path = os.path.relpath('../data/bmc/chamfer_tiles')
bmc_mesh = bmc.marching_cube_mesh(cube_lattice, chamfer_tiles_path)

### 3.4. Visualize the final mesh

In [6]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(tri_to_pv(bmc_mesh), color='#abd8ff', name="sphere")

# fast visualization of the lattice
p = envelope_lattice.fast_vis(p)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(159.11168398125062, 104.11168398125062, 159.11168398125062),
 (30.0, -25.0, 30.0),
 (0.0, 0.0, 1.0)]

### 3.5. Save the final mesh

In [18]:
final_mesh_path = os.path.relpath('../data/final_mesh.obj')

with open(final_mesh_path, 'w') as file:
        file.write(tm.exchange.obj.export_obj(bmc_mesh))

### Credits

In [19]:
__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Polygonization"